In [1]:
import random
from DataPreProcessing.utils.dataprocessutils import extract_data4vfl_2
from mi_estimator import *

In [2]:
features, target, _, _ = extract_data4vfl_2(indice_begin=0, indice_end=len(InitParameters.DATA_LIST))
features = [f.transpose(0, 1, 3, 2) for f in features]
x_dim = np.prod(features[0].shape[1:]).item()
y_dim = np.prod(target.shape[1:]).item()
train_features = [f.reshape(f.shape[0], -1)[:InitParameters.MI_TRAINING_CNT] for f in features]
train_target = target.reshape(target.shape[0], -1)[:InitParameters.MI_TRAINING_CNT]
for cl_index, cl_f in enumerate(train_features):
    # This is the net utilized to calculate the mutual information of two distributions
    mi_est = MINE(input_size=x_dim+y_dim, hidden_size=600).to(device='cuda')
    mi_est_optim = optim.Adam(mi_est.parameters(), lr=2e-4)
    data = np.concatenate((cl_f, train_target), axis=1)
    res = mi_train(data=data, mine_net=mi_est, mine_net_optim=mi_est_optim, x_dim=x_dim)

    model_save_pth = f"..\\checkpoints\\mi_est_{cl_index+1}.pth"
    torch.save(mi_est, model_save_pth)
    print(f"model {cl_index} has been saved to {model_save_pth}!")

2.101979
2.443936
2.7681594
3.1888585


KeyboardInterrupt: 

In [2]:
model_save_pth = "..\\checkpoints\\mi_est_1.pth"

In [3]:
mi_est_1 = torch.load(model_save_pth)

C:\Users\zijun\AppData\Local\Temp\ipykernel_182012\2531528104.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mi_est_1 = torch.load(model_save_pth)


In [2]:
def mi_data_processing(x_data, y_data, sample_count, model_index=0):
    features = [f.transpose(0, 1, 3, 2) for f in x_data]
    x_dim = np.prod(features[0].shape[1:]).item()
    train_features = [f.reshape(f.shape[0], -1) for f in features]
    train_target = y_data.reshape(y_data.shape[0], -1)
    data = np.concatenate((train_features[model_index], train_target), axis=1)
    _data_joint = sample_batch(data, x_dim, sample_count)
    _data_marginal = sample_batch(data, x_dim, sample_count, 'marginal')
    _data_joint = torch.tensor(_data_joint, dtype=torch.float32, device='cuda')
    _data_marginal = torch.tensor(_data_marginal, dtype=torch.float32, device='cuda')

    return _data_joint, _data_marginal

In [5]:
# noise_mean_seq = np.linspace(0, 0.3, 4)
# noise_std_seq = np.linspace(0, 0.3, 4)
noise_mean_seq = np.array([0, 0.01, 0.05, 0.1, 0.2, 0.3])
noise_std_seq = np.array([0, 0.01, 0.05, 0.1, 0.2, 0.3])
print(noise_mean_seq)

[0.   0.01 0.05 0.1  0.2  0.3 ]


In [3]:
model_index = 0
model_save_pth = f"..\\checkpoints\\mi_est_{model_index+1}.pth"
mi_est = torch.load(model_save_pth)

C:\Users\zijun\AppData\Local\Temp\ipykernel_200652\2805449065.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mi_est = torch.load(model_save_pth)


In [16]:
import  csv
import random

sample_time = 1
mi_est.eval()
np.random.seed()
random.seed()
for sample_cnt in range(50, 60, 10):
    mi_res = list()
    for noise_mean in noise_mean_seq:
        mi_res_row = list()
        for noise_std in noise_std_seq:
            features_m, target_m, _, _ = extract_data4vfl_2(indice_begin=0, indice_end=len(InitParameters.DATA_LIST),
                                                num_mal=InitParameters.NUM_PASSIVE_CLIENTS, malicious_flag=1, malicious_exist=True, noise_mean=noise_mean, noise_std=noise_std)
            data_joint, data_marginal = mi_data_processing(features_m, target_m, sample_count=sample_cnt, model_index=model_index)
            mi_out, _, _ = mutual_information(data_joint, data_marginal, mi_est)
            print(f'noise_mean:{noise_mean} \t noise_std:{noise_std} \t sample_cnt:{sample_cnt} \t MI_value:{mi_out.detach().cpu().item()}')
            mi_res_row.append(round(mi_out.detach().cpu().item(), 2))
        mi_res.append(mi_res_row)

    with open(f'..\\checkpoints\\{sample_time}_mal_{model_index+1}_sample_cnt_{sample_cnt}.csv','w',newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(mi_res)

noise_mean:0.0 	 noise_std:0.0 	 sample_cnt:50 	 MI_value:0.25613921880722046
noise_mean:0.0 	 noise_std:0.01 	 sample_cnt:50 	 MI_value:0.29696255922317505
noise_mean:0.0 	 noise_std:0.05 	 sample_cnt:50 	 MI_value:0.3268425166606903
noise_mean:0.0 	 noise_std:0.1 	 sample_cnt:50 	 MI_value:0.068004310131073
noise_mean:0.0 	 noise_std:0.2 	 sample_cnt:50 	 MI_value:-1.0280821323394775
noise_mean:0.0 	 noise_std:0.3 	 sample_cnt:50 	 MI_value:-2.2779088020324707
noise_mean:0.01 	 noise_std:0.0 	 sample_cnt:50 	 MI_value:-0.10763333737850189
noise_mean:0.01 	 noise_std:0.01 	 sample_cnt:50 	 MI_value:-0.03895918279886246
noise_mean:0.01 	 noise_std:0.05 	 sample_cnt:50 	 MI_value:0.1536003053188324
noise_mean:0.01 	 noise_std:0.1 	 sample_cnt:50 	 MI_value:0.1345943808555603
noise_mean:0.01 	 noise_std:0.2 	 sample_cnt:50 	 MI_value:-0.7603466510772705
noise_mean:0.01 	 noise_std:0.3 	 sample_cnt:50 	 MI_value:-2.0681214332580566
noise_mean:0.05 	 noise_std:0.0 	 sample_cnt:50 	 MI_valu

In [13]:
mi_est.eval()

# The data is complete noise generate
for lp in np.linspace(0.2, 1, 5):
    features_b, target_b, _, _ = extract_data4vfl_2(indice_begin=0, indice_end=len(InitParameters.DATA_LIST),
                                                num_mal=InitParameters.NUM_PASSIVE_CLIENTS, malicious_flag=2, malicious_exist=True,lp=lp)
    data_joint_b, data_marginal_b = mi_data_processing(features_b, target_b, sample_count=50, model_index=model_index)
    mi_out_b, _, _ = mutual_information(data_joint_b, data_marginal_b, mi_est)
    print(f'Lazy percentage:{round(lp*100, 1)}% \t MI_value: {mi_out_b.detach().cpu().item()}')

Lazy percentage:20.0% 	 MI_value: -0.1711665391921997
Lazy percentage:40.0% 	 MI_value: -1.888205885887146
Lazy percentage:60.0% 	 MI_value: -3.6094415187835693
Lazy percentage:80.0% 	 MI_value: -4.129734039306641
Lazy percentage:100.0% 	 MI_value: -3.8086647987365723


In [11]:
print(np.linspace(0.2, 1, 5))

[0.2 0.4 0.6 0.8 1. ]


In [6]:
# The data is complete noise generate
sample_cnt = 50
random_data_res = np.zeros(5)
for model_index in range(0, 5):
    model_save_pth = f"..\\checkpoints\\mi_est_{model_index+1}.pth"
    mi_est = torch.load(model_save_pth)
    mi_est.eval()

    i = 0
    for lp in np.linspace(0.2, 1, 5):
        features_b, target_b, _, _ = extract_data4vfl_2(indice_begin=0, indice_end=len(InitParameters.DATA_LIST),
                                                    num_mal=InitParameters.NUM_PASSIVE_CLIENTS, malicious_flag=2, malicious_exist=True,lp=lp)
        data_joint_b, data_marginal_b = mi_data_processing(features_b, target_b, sample_count=50, model_index=model_index)
        mi_out_b, _, _ = mutual_information(data_joint_b, data_marginal_b, mi_est)
        print(f'model_index:{model_index} \t Lazy percentage:{round(lp*100, 1)}% \t MI_value: {round(mi_out_b.detach().cpu().item(),2)}')
        random_data_res[i] += round(mi_out_b.detach().cpu().item() / 5,2)
        i += 1

print(random_data_res)

C:\Users\zijun\AppData\Local\Temp\ipykernel_201920\3048734217.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mi_est = torch.load(model_save_pth)


model_index:0 	 Lazy percentage:20.0% 	 MI_value: -0.68
model_index:0 	 Lazy percentage:40.0% 	 MI_value: 0.06
model_index:0 	 Lazy percentage:60.0% 	 MI_value: -1.73
model_index:0 	 Lazy percentage:80.0% 	 MI_value: -4.29
model_index:0 	 Lazy percentage:100.0% 	 MI_value: -3.81
model_index:1 	 Lazy percentage:20.0% 	 MI_value: 1.14
model_index:1 	 Lazy percentage:40.0% 	 MI_value: -2.94
model_index:1 	 Lazy percentage:60.0% 	 MI_value: -6.79
model_index:1 	 Lazy percentage:80.0% 	 MI_value: -5.9
model_index:1 	 Lazy percentage:100.0% 	 MI_value: -10.42
model_index:2 	 Lazy percentage:20.0% 	 MI_value: -2.28
model_index:2 	 Lazy percentage:40.0% 	 MI_value: -1.97
model_index:2 	 Lazy percentage:60.0% 	 MI_value: -6.06
model_index:2 	 Lazy percentage:80.0% 	 MI_value: -5.43
model_index:2 	 Lazy percentage:100.0% 	 MI_value: -8.26
model_index:3 	 Lazy percentage:20.0% 	 MI_value: -3.44
model_index:3 	 Lazy percentage:40.0% 	 MI_value: -3.21
model_index:3 	 Lazy percentage:60.0% 	 MI_valu

In [7]:
sample_cnt = 50
historical_data_res = np.zeros(5)
for model_index in range(0, 5):
    model_save_pth = f"..\\checkpoints\\mi_est_{model_index+1}.pth"
    mi_est = torch.load(model_save_pth)
    mi_est.eval()

    i = 0
    for lp in np.linspace(0.2, 1, 5):
        features_b, target_b, _, _ = extract_data4vfl_2(indice_begin=0, indice_end=len(InitParameters.DATA_LIST),
                                                    num_mal=InitParameters.NUM_PASSIVE_CLIENTS, malicious_flag=3, malicious_exist=True,lp=lp)
        data_joint_b, data_marginal_b = mi_data_processing(features_b, target_b, sample_count=50, model_index=model_index)
        mi_out_b, _, _ = mutual_information(data_joint_b, data_marginal_b, mi_est)
        print(f'model_index:{model_index} \t Lazy percentage:{round(lp*100, 1)}% \t MI_value: {round(mi_out_b.detach().cpu().item(),2)}')
        historical_data_res[i] += round(mi_out_b.detach().cpu().item() / 5,2)
        i += 1

print(historical_data_res)

C:\Users\zijun\AppData\Local\Temp\ipykernel_201920\3156154437.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mi_est = torch.load(model_save_pth)


model_index:0 	 Lazy percentage:20.0% 	 MI_value: -1.09
model_index:0 	 Lazy percentage:40.0% 	 MI_value: -0.87
model_index:0 	 Lazy percentage:60.0% 	 MI_value: -3.57
model_index:0 	 Lazy percentage:80.0% 	 MI_value: -5.22
model_index:0 	 Lazy percentage:100.0% 	 MI_value: -5.51
model_index:1 	 Lazy percentage:20.0% 	 MI_value: 0.64
model_index:1 	 Lazy percentage:40.0% 	 MI_value: -1.89
model_index:1 	 Lazy percentage:60.0% 	 MI_value: -5.65
model_index:1 	 Lazy percentage:80.0% 	 MI_value: -7.51
model_index:1 	 Lazy percentage:100.0% 	 MI_value: -7.11
model_index:2 	 Lazy percentage:20.0% 	 MI_value: -1.13
model_index:2 	 Lazy percentage:40.0% 	 MI_value: -1.98
model_index:2 	 Lazy percentage:60.0% 	 MI_value: -4.27
model_index:2 	 Lazy percentage:80.0% 	 MI_value: -5.71
model_index:2 	 Lazy percentage:100.0% 	 MI_value: -9.09
model_index:3 	 Lazy percentage:20.0% 	 MI_value: 1.2
model_index:3 	 Lazy percentage:40.0% 	 MI_value: -7.04
model_index:3 	 Lazy percentage:60.0% 	 MI_value